In [4]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
import numpy as np
from math import pi, sqrt
import random

In [6]:
# the basis gates
basis_g = ['id', 'rz', 'sx', 'x', 'cx']

In [18]:
# single qubit gates with no parameters
sgle_noparam = ["i","x","y","z","h","s","sdg","t","tdg","sx","sxdg"]

for gate in sgle_noparam:
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    print(transpile_circuit.size())

0
1
2
1
3
1
1
1
1
1
3


In [19]:
# single qubit gates with 1 parameter
sgle_1_param = ["p","rx","ry","rz"]

for gate in sgle_1_param:
    param = random.uniform(-np.pi, np.pi)
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(param,0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    print(transpile_circuit.size())

1
5
4
1


In [20]:
# single qubit gates with 2 parameters
single_2_param = ["u2"]

for gate in single_2_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-np.pi, np.pi)
    rand_angle2 = random.uniform(-np.pi, np.pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    print(transpile_circuit.size())


3


/tmp/ipykernel_131/684528647.py:8: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, 0)


In [21]:
# single qubit gates with 3 parameters
single_3_param = ["u", "u3"]

for gate in single_3_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-pi, pi)
    rand_angle2 = random.uniform(-pi, pi)
    rand_angle3 = random.uniform(-pi, pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    print(transpile_circuit.size())

5
5


/tmp/ipykernel_131/2798487310.py:9: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)
